In [ ]:
import os
import json
import numpy as np

In [ ]:

# ==========================================
# STEP 1: Tentukan direktori model yang akan dianalisis
# ==========================================
MODEL_DIR = "../models"
MODEL_SUBDIR = "model2-1-93-86"   # 🔹 ubah sesuai folder model yang mau dibaca
model_path = os.path.join(MODEL_DIR, MODEL_SUBDIR)

summary_path = os.path.join(model_path, "summary.json")
eval_val_path = os.path.join(model_path, "evaluation_val.json")
eval_test_path = os.path.join(model_path, "evaluation_test.json")

# ==========================================
# STEP 2: Baca file JSON
# ==========================================
with open(summary_path, "r") as f:
    summary = json.load(f)

with open(eval_val_path, "r") as f:
    report_val = json.load(f)

with open(eval_test_path, "r") as f:
    report_test = json.load(f)


In [49]:

# ==========================================
# STEP 3: Analisis metrik dari file JSON
# ==========================================
# Ambil daftar kelas (tanpa 'accuracy', 'macro avg', 'weighted avg')
class_labels = [k for k in report_test.keys() if k not in ("accuracy", "macro avg", "weighted avg")]

val_precision = np.mean([report_val[c]["precision"] for c in class_labels])
test_precision = np.mean([report_test[c]["precision"] for c in class_labels])
test_recall = np.mean([report_test[c]["recall"] for c in class_labels])
test_f1 = np.mean([report_test[c]["f1-score"] for c in class_labels])

# ==========================================
# STEP 4: Cetak hasil & kesimpulan
# ==========================================
print("\n📊 HASIL ANALISIS MODEL:")
print(f"• Model Name      : {summary['model_name']}")
print(f"• Version         : {summary['version']}")
print(f"• Train Accuracy  : {summary['train_acc_avg']:.2f}%")
print(f"• Val Accuracy    : {summary['val_acc_avg']:.2f}%")
print(f"• Test Accuracy   : {summary['test_accuracy']:.2f}%")
print(f"• Test Loss       : {summary['test_loss']:.4f}")
print(f"• Precision (avg) : {test_precision:.2%}")
print(f"• Recall (avg)    : {test_recall:.2%}")
print(f"• F1-score (avg)  : {test_f1:.2%}")



📊 HASIL ANALISIS MODEL:
• Model Name      : MobileNetV2
• Version         : 1
• Train Accuracy  : 93.24%
• Val Accuracy    : 86.74%
• Test Accuracy   : 92.42%
• Test Loss       : 0.6040
• Precision (avg) : 66.39%
• Recall (avg)    : 62.28%
• F1-score (avg)  : 63.73%


In [50]:
# ==========================================
# STEP 5: Kesimpulan otomatis (SMART ANALYSIS)
# ==========================================
print("\n=== KESIMPULAN OTOMATIS ===")

acc = summary["test_accuracy"]
train_acc = summary["train_acc_avg"]
val_acc = summary["val_acc_avg"]

# ------------------------------
# Interpretasi Accuracy
# ------------------------------
if acc >= 90:
    acc_msg = "Akurasi sangat tinggi → model sangat baik dalam mengenali mayoritas data."
elif acc >= 80:
    acc_msg = "Akurasi baik → model cukup stabil namun masih bisa ditingkatkan."
elif acc >= 70:
    acc_msg = "Akurasi sedang → model belum konsisten dalam prediksi."
else:
    acc_msg = "Akurasi rendah → model belum mampu mengenali pola dengan baik."

# ------------------------------
# Interpretasi Precision
# ------------------------------
if test_precision >= 0.85:
    prec_msg = "Precision tinggi → kesalahan False Positive sangat kecil."
elif test_precision >= 0.70:
    prec_msg = "Precision cukup → masih ada beberapa False Positive."
else:
    prec_msg = "Precision rendah → model sering salah memprediksi kelas (False Positive tinggi)."

# ------------------------------
# Interpretasi Recall
# ------------------------------
if test_recall >= 0.85:
    recall_msg = "Recall tinggi → model jarang melewatkan objek penting."
elif test_recall >= 0.70:
    recall_msg = "Recall cukup → masih ada beberapa objek yang tidak terdeteksi."
else:
    recall_msg = "Recall rendah → banyak objek penting terlewat (False Negative tinggi)."

# ------------------------------
# Interpretasi F1 Score
# ------------------------------
if test_f1 >= 0.85:
    f1_msg = "F1-score sangat baik → keseimbangan Precision & Recall sangat bagus."
elif test_f1 >= 0.70:
    f1_msg = "F1-score cukup → model lumayan stabil namun belum optimal."
else:
    f1_msg = "F1-score rendah → model belum seimbang dalam klasifikasi tiap kelas."

# ------------------------------
# Analisis Overfitting
# ------------------------------
gap = train_acc - val_acc
if gap < 5:
    overfit_msg = "Tidak ada indikasi overfitting → generalisasi model baik."
elif gap < 10:
    overfit_msg = "Sedikit overfitting → model mulai terlalu hafal data training."
else:
    overfit_msg = "Overfitting cukup besar → model kurang generalisasi."

# ------------------------------
# Analisis Keseimbangan Model
# ------------------------------
if acc > 90 and test_f1 < 0.70:
    balance_msg = (
        "Akurasi tinggi tetapi F1 rendah → kemungkinan dataset tidak seimbang "
        "atau model bias ke kelas mayoritas."
    )
elif test_precision < 0.70 and test_recall < 0.70:
    balance_msg = "Precision & Recall rendah → model masih sering salah klasifikasi."
else:
    balance_msg = "Keseimbangan model cukup baik."

# ------------------------------
# Gabungkan Kesimpulan
# ------------------------------
conclusion = (
    f"{acc_msg}\n"
    f"{prec_msg}\n"
    f"{recall_msg}\n"
    f"{f1_msg}\n"
    f"{overfit_msg}\n"
    f"{balance_msg}"
)

print("# Analisis Model:")
print(conclusion)



=== KESIMPULAN OTOMATIS ===
# Analisis Model:
Akurasi sangat tinggi → model sangat baik dalam mengenali mayoritas data.
Precision rendah → model sering salah memprediksi kelas (False Positive tinggi).
Recall rendah → banyak objek penting terlewat (False Negative tinggi).
F1-score rendah → model belum seimbang dalam klasifikasi tiap kelas.
Sedikit overfitting → model mulai terlalu hafal data training.
Akurasi tinggi tetapi F1 rendah → kemungkinan dataset tidak seimbang atau model bias ke kelas mayoritas.


In [51]:

# ==========================================
# STEP 6: (Opsional) Simpan kesimpulan ke summary.json
# ==========================================
summary["final_conclusion"] = conclusion
summary["test_precision_avg"] = round(float(test_precision) * 100, 2)
summary["test_recall_avg"] = round(float(test_recall) * 100, 2)
summary["test_f1_avg"] = round(float(test_f1) * 100, 2)

with open(summary_path, "w") as f:
    json.dump(summary, f, indent=4)

print(f"\n✅ Kesimpulan diperbarui ke file: {summary_path}")



✅ Kesimpulan diperbarui ke file: ../model\model2-1-93-86\summary.json
